In [1]:
import pandas as pd
import numpy as np

## Veri setini okuma ve ön işleme

In [2]:
df1=pd.read_csv("resumes_dataset/resumes_development.csv",dtype=int)
df2=pd.read_csv("resumes_dataset/resumes_pilot.csv",dtype=int)
data=pd.concat([df1,df2],ignore_index=True)

Yapay zekanın önyargılı karar vermesini engellemek amacıyla aşağıdaki sütunları çıkardım.  

In [3]:
data.drop(["Unnamed: 0","Female","URM","Disability"],axis=1,inplace=True)

bağımlı(y) ve bağımsız(x) değişkenlerini oluşturma

In [4]:
x=data.iloc[:,1:220].values
y=data["Interview"].values.reshape(-1,1)

x ve y verilerini, test ve eğitim seti olarak ikiye bölme

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=46)

## LogisticRegression ile 1. modeli oluşturma ve eğitim verisiyle eğitme

In [7]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()

In [ ]:
lr.fit(x_train,y_train)

## 1. modeli test etme

In [9]:
y_head=lr.predict(x_test)

In [10]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_head)
cm

array([[517,   8],
       [ 10, 247]], dtype=int64)

In [11]:
print(lr.score(x_test,y_test))
print(lr.score(x_train,y_train))

0.9769820971867008
0.9923203510696654


## Overfitting'i (aşırı öğrenme) önlemek için C=0.1 alarak modeli tekrar eğitme

In [ ]:
lr1=LogisticRegression(C=0.1)
lr1.fit(x_train,y_train)

## Yeni oluşturduğum modeli test etme

In [13]:
print(lr1.score(x_test,y_test))
print(lr1.score(x_train,y_train))

0.9616368286445013
0.9709270433351618


In [14]:
y_head1=lr1.predict(x_test)

In [15]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_head1)
cm

array([[515,  10],
       [ 20, 237]], dtype=int64)

Cross validation skorlarını elde etme

In [16]:
lr2=LogisticRegression(C=0.1)

In [17]:
column=[]
for i in range(1,11):
    column.append("Model"+str(i))
column.append("Avarage")
models=pd.DataFrame(columns=column)

In [18]:
from sklearn.model_selection import cross_val_score

In [ ]:
for i in range(4,11):
    c_score=cross_val_score(lr2,x,y,cv=i)
    avarage=c_score.mean()
    c_score=c_score.tolist()
    for j in range(i,10):
        c_score.append(None)
    c_score.append(avarage)    
    models.loc[i]=c_score

In [20]:
models

,Model1,Model2,Model3,Model4,Model5,Model6,Model7,Model8,Model9,Model10,Avarage
4,0.967791,0.958525,0.967742,0.960061,NaN,NaN,NaN,NaN,NaN,NaN,0.963530
5,0.967370,0.965451,0.963532,0.965451,0.955854,NaN,NaN,NaN,NaN,NaN,0.963532
6,0.967816,0.970046,0.960829,0.970046,0.958525,0.960829,NaN,NaN,NaN,NaN,0.964682
7,0.962466,0.973118,0.970430,0.956989,0.965054,0.956989,0.962366,NaN,NaN,NaN,0.963916
8,0.963190,0.972393,0.969325,0.953988,0.969325,0.972308,0.947692,0.963077,NaN,NaN,0.963912
9,0.962069,0.965517,0.975862,0.962069,0.965398,0.968858,0.968858,0.944637,0.965398,NaN,0.964296
10,0.961686,0.969349,0.969349,0.969349,0.954023,0.976923,0.969231,0.965385,0.942308,0.969231,0.964683


KFold ile Cross validation skorlarını elde etme

In [21]:
column=[]
for i in range(1,11):
    column.append("Model"+str(i))
column.append("Avarage")
models=pd.DataFrame(columns=column)

In [22]:
from sklearn.model_selection import KFold

In [ ]:
kfold=KFold(n_splits=5,shuffle=True,random_state=0)
for i in range(4,11):
    kfold=KFold(n_splits=i,shuffle=True,random_state=0)
    kfold_c_score=cross_val_score(lr2,x,y,cv=kfold)
    avarage=kfold_c_score.mean()
    kfold_c_score=kfold_c_score.tolist()
    for j in range(i,10):
        kfold_c_score.append(None)
    kfold_c_score.append(avarage)    
    models.loc[i]=kfold_c_score

In [24]:
models

,Model1,Model2,Model3,Model4,Model5,Model6,Model7,Model8,Model9,Model10,Avarage
4,0.966258,0.958525,0.967742,0.973886,NaN,NaN,NaN,NaN,NaN,NaN,0.966603
5,0.965451,0.957774,0.963532,0.963532,0.973129,NaN,NaN,NaN,NaN,NaN,0.964683
6,0.963218,0.972350,0.953917,0.970046,0.965438,0.970046,NaN,NaN,NaN,NaN,0.965836
7,0.959786,0.975806,0.948925,0.956989,0.965054,0.978495,0.970430,NaN,NaN,NaN,0.965069
8,0.957055,0.978528,0.950920,0.963190,0.969325,0.963077,0.975385,0.966154,NaN,NaN,0.965454
9,0.955172,0.979310,0.968966,0.941379,0.972318,0.968858,0.961938,0.972318,0.968858,NaN,0.965458
10,0.950192,0.980843,0.973180,0.946360,0.954023,0.969231,0.961538,0.969231,0.976923,0.969231,0.965075


## Modeli kaydetme

In [25]:
import pickle

In [26]:
pickle.dump(lr1,open("lr_model.pickle","wb"))